In [33]:
import matplotlib.pyplot as plt
import numpy as np
import math
import decimal

In [34]:
def draw(Ox, Oy: list[list], name: str, scatter: bool = False):
    f = plt.figure(figsize=(7, 7))
    px1 = f.add_subplot(111)
    if not scatter:
        for y in Oy:
            px1.plot(Ox, y, label=name)
    else:
        for y in Oy:
            px1.scatter(Ox, y, label=name)
    px1.legend()
    f.show()

In [35]:
def f(ksi, a, c, T1, T2):
    if 0 < ksi <= T1:
        return a
    elif T1 < ksi <= T2:
        return ((a - c) * ksi  + c * T1 - a * T2)/(T1 - T2)
    else:
        return ((c - a) * ksi  + a * T2  - c) / (T2 - 1)

def fx(xvn, yvn, a):
    # print(xvn, yvn)
    # return -a*((xvn**3)/3-xvn)+a*yvn
    return yvn

# def fy(xvn, yvn, c, b):
def fy(xvn, yvn, c, b, T1, T2, a, t, Tbig, K, eps, Tau, arrayx, T):
    # return -xvn + c - b*yvn
    print(yvn * (f(t/Tbig + 1/4, a, c, T1, T2) - xvn ** 2) - xvn + K + eps * (arrayx[-(Tau/2/T)] - xvn), " || ", xvn)
    print( )
    return yvn * (f(t/Tbig + 1/4, a, c, T1, T2) - xvn ** 2) - xvn + K + eps * (arrayx[-(Tau/2/T)] - xvn)
    # return -xvn

In [36]:
def newtonsMethod(a, c, T, y, b, x):
    l = [x]
    for i in range(100):
        xn = l[-1]
        f = -xn+(-a*(xn**3/3-xn)+a*((-xn+c)*T+y)/(1+b*T))*T+x
        g = -1+a*T*(-T/(b*T+1)-xn**2+1)
        l.append(l[-1] - f/g)
    return l[-1]

In [37]:
# def explicit_method(T, a, xvn, yvn, c, b):
def explicit_method(T, a, xvn, yvn, c, b, T1, T2, t, Tbig, K, eps, Tau, arrayx):
    xn1 = T*fx(xvn, yvn, a) + xvn
    # yn1 = T*fy(xvn, yvn, c, b) + yvn
    yn1 = T*fy(xvn, yvn, c, b, T1, T2, a, t, Tbig, K, eps, Tau, arrayx, T) + yvn
    return (xn1, yn1)

In [38]:
def implicit_method(T, a, xvn, yvn, c, b):
    xn2 = newtonsMethod(a, c, T, yvn, b, xvn)
    yn2 = (T*(-xn2+c) + yvn)/(1+b*T)
    return (xn2, yn2)

In [39]:
def explicit4_method(T, a, xvn, yvn, c, b):
    kx1 = fx(xvn, yvn, a)
    kx2 = fx(xvn + T/2*kx1, yvn + T/2, a)
    kx3 = fx(xvn + T/2*kx2, yvn + T/2, a)
    kx4 = fx(xvn + T*kx3, yvn + T, a)
    xn = xvn + T/6*(kx1 + 2 * kx2 + 2 * kx3 + kx4)

    ky1 = fy(xvn, yvn, c, b)
    ky2 = fy(xvn + T/2, yvn + T/2 * ky1, c, b)
    ky3 = fy(xvn + T/2, yvn + T/2 * ky2, c, b)
    ky4 = fy(xvn + T/2, yvn + T * ky3, c, b)
    yn = yvn + T/6*(ky1 + 2 * ky2 + 2 * ky3 + ky4)
    
    return (xn, yn)

In [40]:
def explicit5_method(T, a, xvn, yvn, c, b):
    kx1 = fx(xvn, yvn, a)
    kx2 = fx(xvn + 1/4 * T * kx1,
             yvn + 3/2 * T, a)
    # if xvn + (3/32 * kx1 + 9/32 * kx2) * T > 10:
    #     print(xvn + (3/32 * kx1 + 9/32 * kx2) * T)
    kx3 = fx(xvn + (3/32 * kx1 + 9/32 * kx2) * T,
             yvn + 3/8 * T, a)
    kx4 = fx(xvn + (1932/2197 * kx1 - 
             7200/2197 * kx2 + 7296/2197 * kx3) * T,
             yvn + 12/13 * T, a)
    kx5 = fx(xvn + (439/216 * kx1 - 8 * kx2
                    + 3680/513 * kx3 - 845/4104 * kx4) * T,
                    yvn + T, a)
    kx6 = fx(xvn + (-8/27 * kx1 + 2 * kx2 - 3544/2565 * kx3
                    + 1859/4104 * kx4 + 11/40 * kx5) * T,
                    yvn + 1/2 * T, a)
    xn = xvn + (16/135 * kx1 + 6656/12825 * kx3
                + 28561/56430 * kx4 + 2/55 * kx5
                - 9/50 * kx6) * T
    
    ky1 = fy(xvn, yvn, c, b)
    ky2 = fy(xvn + 3/2 * T,
             yvn + 1/4 * T * ky1, c, b)
    ky3 = fy(xvn + 3/8 * T,
             yvn + (3/32 * ky1 + 9/32 * ky2) * T, c, b)
    ky4 = fy(xvn + 12/13 * T, yvn + (1932/2197 * ky1 - 
            7200/2197 * ky2 + 7296/2197 * ky3) * T, c, b)
    ky5 = fy(xvn + T, yvn + (439/216 * ky1 - 8 * ky2
             + 3680/513 * ky3 - 845/4104 * ky4) * T, c, b)
    ky6 = fy(xvn + 1/2 * T,
             yvn + (-8/27 * ky1 + 2 * ky2 - 3544/2565 * ky3
             + 1859/4104 * ky4 + 11/40 * ky5) * T, c, b)
    yn = yvn + (16/135 * ky1 + 6656/12825 * ky3
                + 28561/56430 * ky4 + 2/55 * ky5
                - 9/50 * ky6) * T

    return (xn, yn)

$x' = -a(\frac{x^3}{3} - x) - ay$\
$y' = -x - by + c$\
$x(0) = 2,\quad y(0) = 0$\
Здесь 1 $\leq$ _a_ $\leq$ 10^3, 0 < _c_ < 1.
Провести исследование поведения решений в зависимости от значений «большого» параметра _a_.


In [41]:
T = 30

K = 0.5
c = -2
T1 = 0.4
T2 = 0.5
eps = 0.01
Tbig = 200
Tau = Tbig/2

t_start = 18 * Tbig
t_finish = 20 * Tbig

# for a in range(1, 1000, 100):
    # b /= 1000
# for c in range(1, 100, 30):
    # c /= 10
    # for b in range(1, 200, 10):
a = 5.5
b = 0
tn = t_start
# Первый элемент это x, второй y
# array1 = [(2, 0)]
# array2 = [(2, 0)]
main_array = []
tl = [tn]
while tl[-1] <= t_finish:
    tl.append(tl[-1] + T)
iter = tl[1:]

# c = 0.5
# a = 100
array1 = [(2, 0)]
array2 = [(2, 0)]
array3 = [(2, 0)]
array4 = [(2, 0)]

# array3_dif = [(fx(array3[-1][0], array3[-1][1], a),
#                     fy(array3[-1][0], array3[-1][1], c, b))]

for i in range(1, len(tl)):
    t = tl[i]
# for _ in iter:
    #явный метод 1-го порядка
    # array1.append(explicit_method(T, a, array1[-1][0], array1[-1][1], c, b, T1, T2, t, Tbig, K, eps, Tau))
    array1.append((fx(array1[-1][0], array1[-1][1], a), fy(array1[-1][0], array1[-1][1], c, b, T1, T2, a, t, Tbig, K, eps, Tau)))
    #неявный метод 1-го порядка
    # array2.append(implicit_method(T, a, array2[-1][0], array2[-1][1], c, b))
    #метод рунге-кутты 4-го порядка
    # array3.append(explicit4_method(T, a, array3[-1][0], array3[-1][1], c, b))
    # array4.append(explicit5_method(T, a, array4[-1][0], array4[-1][1], c, b))
    
    # array3_dif.append((fx(array3[-1][0], array3[-1][1], a),
    #                 fy(array3[-1][0], array3[-1][1], c, b)))

arrayx1 = [i[0] for i in array1]
# arrayx2 = [i[0] for i in array2]
arrayx3 = [i[0] for i in array3]
arrayx4 = [i[0] for i in array4]

# main_array.append(arrayx3)
# array_dif_x3 = [[i[0] for i in array3_dif]]
# draw(px1, tl, arrayx1, "явный 1-го порядка")
# draw(px1, tl, arrayx2, "неявный 1-го порядка")
draw(tl, [arrayx1], f"{a}, {c} рунге-кутты")
# draw(tl, array_dif_x3, f"{a} x' from t")
# draw(arrayx3, array_dif_x3, "x' from x")

69.08  ||  2

18565.75  ||  0

-83562891.4996  ||  69.08

2.8803025654729612e+16  ||  18565.75

-2.011245242815269e+32  ||  -83562891.4996

1.6685577878328602e+65  ||  2.8803025654729612e+16

-6.749495499519331e+129  ||  -2.011245242815269e+32

1.8791169794262595e+260  ||  1.6685577878328602e+65

-inf  ||  -6.749495499519331e+129



OverflowError: (34, 'Numerical result out of range')